In [8]:
import pandas as pd
import json
import numpy as np
import requests
from config import (client_id, api_key)
from pprint import pprint
import pydbgen
from pydbgen import pydbgen
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [11]:
business_data={"Business ID":[],
               "Business Name":[],
               "Business Alias":[],
               "Category":[],
               "City":[],
               "State":[],
               "ZipCode":[],
               "Lat":[],
               "Long":[],
               "Review Count":[],
               "Rating":[],
              }
review_data={"Business ID":[],
             "Review":[],
            }

In [14]:
myDB=pydbgen.pydb()
city=[]
for cities in range(500):
    city.append(myDB.city_real())
city_yelp=pd.DataFrame(city)
city_yelp=city_yelp.rename(columns={0:"US Cities"})
city_yelp.head()

,US Cities
0,Brockton
1,Horse Branch
2,Duque
3,Green Hill
4,Red Banks


In [17]:
api_key=api_key
headers = {"Authorization": (f"Bearer {api_key}")}
url="https://api.yelp.com/v3/businesses/search?"
review_url="https://api.yelp.com/v3/businesses/"
categories="lawyers"
radius = 4000
counter = 1

In [18]:
for index, row in city_yelp.iterrows():
    try:
        random_city = row['US Cities']
        print(f"{counter}: {random_city}")
        yelp_response2 = requests.get((f"{url}categories={categories}&location={random_city}&radius={radius}"), headers=headers).json()
        #pprint(yelp_response2)
        counter+=1
    except:
        print(f"Could not locate in lawyers in {random_city}.")
    #print(len(yelp_response2["businesses"]))
    for x in range(len(yelp_response2["businesses"])):
        try:
            business_data["Business ID"].append(yelp_response2["businesses"][x]["id"])
            business_data["Business Name"].append(yelp_response2["businesses"][x]["name"])
            business_data["Business Alias"].append(yelp_response2["businesses"][x]["alias"])
            business_data["Category"].append(yelp_response2["businesses"][x]["categories"][0]["alias"])
            business_data["City"].append(yelp_response2["businesses"][x]["location"]["city"])
            business_data["State"].append(yelp_response2["businesses"][x]["location"]["state"])
            business_data["ZipCode"].append(yelp_response2["businesses"][x]["location"]["zip_code"])
            business_data["Lat"].append(yelp_response2["businesses"][x]["coordinates"]["latitude"])
            business_data["Long"].append(yelp_response2["businesses"][x]["coordinates"]["longitude"])
            business_data["Review Count"].append(yelp_response2["businesses"][x]["review_count"])
            business_data["Rating"].append(yelp_response2["businesses"][x]["rating"])
        except: 
            print(f"     No dining locations found in {random_city}.")

1: Brockton
2: Horse Branch
3: Duque
4: Green Hill
5: Red Banks
6: Stayton
7: Sumpter
8: Ethelsville
9: West Boxford
10: Spring Mill
11: Gerty
12: Brazos Country
13: Pineola
14: Perry
15: Pentwater
16: Zita
17: Timpas
18: Hornitos
19: Inglewood
20: Montana City
21: Garberville
22: Norton
23: Bryceland
24: Grotto
25: Yanush
26: Halfa
27: Paxtang
28: Concordia
29: Caddo Mills
30: Winter Beach
31: Myrtle Springs
32: Mountain View Acres
33: Chatsworth
34: Gnadenhutten
35: Ledger
36: Huntington Station
37: Gallipolis
38: Christoval
39: Aransas Pass
40: Berea
41: Gonzales
42: Two Inlets
43: Lake Providence
44: Brushvale
45: Teutopolis
46: Patton
47: Luverne
48: East Rockaway
49: Moose Creek
50: Palestine
51: Harpersville
52: Wesleyan
53: Orchard Grass Hills
54: Whittaker
55: Wanatah
56: Lavaca
57: Hutsonville
58: Ridgeway
59: Pleasant Site
60: Armbrust
61: Kinter
62: Progreso
63: Wheeler Ridge
64: Harding Lake
65: Shiner
66: Dorcas
67: Brookshire
68: West View
69: Audubon Park
70: Benevolenc

In [19]:
#pprint(yelp_response2)

In [20]:
#business_data

In [21]:
graph_data=pd.DataFrame.from_dict(business_data, orient='columns', dtype=None, columns=None)

In [22]:
graph_data["Review Count"].sum()

20132

In [23]:
for id in business_data["Business ID"]:
    try:
        yelp_response3 = requests.get((f"{review_url}{id}/reviews"), headers=headers).json()
        #pprint(yelp_response3)
        print(f"Fetching Reviews for {id}.")
    except:
        print(f"No Reviews for {id}.")
    for x in range(len(yelp_response3["reviews"])):
        try:
            review_data["Business ID"].append(id)
            review_data["Review"].append(yelp_response3["reviews"][x]["text"])
        except:
            print("skipping")

Fetching Reviews for LVdPec0oNU0xm5RBzcsDTQ.
Fetching Reviews for 6gan8fxBSkGQJ7vLsK0ENQ.
Fetching Reviews for WcFu0Esq3cm-hKHichMcXg.
Fetching Reviews for 6zj8agbUy2kdJElqSNProQ.
Fetching Reviews for jYYzEiMr-8SZqXzFVjd2wg.
Fetching Reviews for _KOFlsdPKZ7eFG3gBgEhzQ.
Fetching Reviews for 3r3AMohYkc_MLDhtahwtng.
Fetching Reviews for _n1EsK4YBJqgiB-0yEuLEg.
Fetching Reviews for EWfsI4Jet5r9W1CFK9DFLA.
Fetching Reviews for IKI4yRT587EcClz8yH309A.
Fetching Reviews for F5BrO-07gcwZ00GGLHLoHQ.
Fetching Reviews for voo5l0HSWGyX-MGFM0g8DA.
Fetching Reviews for Fnn-s8riqsXgouaVGWeoEg.
Fetching Reviews for 6d_2IJ7674jfaLfN4vp0ZQ.
Fetching Reviews for -NFStDaoEEM0bHsuhvZs5w.
Fetching Reviews for 92IKMmkddLfkz7FMZj-A9g.
Fetching Reviews for kla1dKKlE5TtNzA_CBUJgg.
Fetching Reviews for KcCwXYkZVQRU7AUuB671Vw.
Fetching Reviews for VDCeCrCrT1Qa24pwPLpGpQ.
Fetching Reviews for GISdGW70DbYZHUDt7H7p1g.
Fetching Reviews for vVqZ-io4zTCouAEQ6IgIOA.
Fetching Reviews for 7T8tW_gSK9TmD9G8IXnOxA.
Fetching R

Fetching Reviews for EXgHPxB_jgElBtjHAv8wmg.
Fetching Reviews for JZcxnokkRBPRIDJUXcUpVQ.
Fetching Reviews for fg9hxgIAuoak0xjDQDkBNw.
Fetching Reviews for -Cde1kHfS69Ms-pcOoN_tg.
Fetching Reviews for -hVtH-WY2JjPFRP666ZAAA.
Fetching Reviews for 8UDRNefL_wCqsahp97THTA.
Fetching Reviews for Gy9Jjht9NjWfj10nXCYK9A.
Fetching Reviews for Ieep_J2_byXO2WB-oYY31g.
Fetching Reviews for PCzLQ75Tc_VFSicncCJsiQ.
Fetching Reviews for Jx11hlbBgMEi42oOI1oUjQ.
Fetching Reviews for tbGpNhFsTTWxi80oJ3vs2g.
Fetching Reviews for pWbwNzEQEEf1RMqAYdwWuA.
Fetching Reviews for WL54cIWmck0U5UeUwVjI2g.
Fetching Reviews for q3FBX0VunQSH7yypw3RGPQ.
Fetching Reviews for ESD4aXe18K8nVTU9Y4BbKg.
Fetching Reviews for ZaQssq23pP59HRrRdoW8bA.
Fetching Reviews for 4ZvVLkBM5YqXVl-DU10SaA.
Fetching Reviews for 40sFSMe9uKma8ZiXhBbVFg.
Fetching Reviews for TBO8VTe_i7JNs134n9qYhw.
Fetching Reviews for x__i__T_wNUNF2eT1ZFFxg.
Fetching Reviews for 4T67lZ9zeCvn4bnrhy6cbA.
Fetching Reviews for Ctjlm_8-wibYXFvei_9J8A.
Fetching R

Fetching Reviews for mFJoSV4BMbbnZR4kAdCivA.
Fetching Reviews for cE9si0wQrcfMjC5rHyH4Jw.
Fetching Reviews for dWK6jz_f4E0wF9etZVDx3Q.
Fetching Reviews for uUsPev8g6ZsQu297RP1rxg.
Fetching Reviews for 5whK41n1d__67Z3iPWC0tQ.
Fetching Reviews for jQEg3MNnqjON787VAbSmyQ.
Fetching Reviews for qHqhheSva82fIcddIIH4VQ.
Fetching Reviews for hSJSuJGvQ-ZjER7EHmhRWg.
Fetching Reviews for MH61SVerBTjdDcnGAEBPtQ.
Fetching Reviews for 7koSJgtGo0Mcpw71LGw2Eg.
Fetching Reviews for LlUSYGupMRI2uPcp5WQxyA.
Fetching Reviews for 7ccuIk55N8AI84R706OZvA.
Fetching Reviews for 0Z1KpaAORu520wT6TfOAmQ.
Fetching Reviews for uctowFL25FWM3D310IpbjQ.
Fetching Reviews for xeaBOlEwbq1QOvx6xvuVHg.
Fetching Reviews for G6IK_51qt9jQ_gRT8adepg.
Fetching Reviews for R1VJTJcIpqhIC8Pd2CLHMA.
Fetching Reviews for wyXKvZRWuy2tSYl5gjHAPg.
Fetching Reviews for DeKrXizqMkn5kgTk-bZ-OA.
Fetching Reviews for qNSAcnBP_eRhU8yQnt-A7A.
Fetching Reviews for RD63v6jCCi0XvXKT-I1eXQ.
Fetching Reviews for qHg5S9Ym8PaRRa9PMRBjQg.
Fetching R

Fetching Reviews for b3NhTExih55tSv7FZ6Y4fg.
Fetching Reviews for XNjv9XSRMypUvwWCCKYN5Q.
Fetching Reviews for Ktp6RRhk8sLJo6BQ9ZIG6g.
Fetching Reviews for kSeRT589KT1x4c1merLZDg.
Fetching Reviews for SUa6eh1BMoUDR7Ixdv3ckA.
Fetching Reviews for zmlpbMuVWHB176ACDRZbww.
Fetching Reviews for Za115lgH6GhrfpfVlFK7hA.
Fetching Reviews for liq6QD-9Y5yMsicT5MgEDw.
Fetching Reviews for AyAOEsHz_w0_p33tgcK-XA.
Fetching Reviews for wd1pQ3DYgsQF_5J1L5tevw.
Fetching Reviews for -OxL4hy8QZpwusXXxlSEPA.
Fetching Reviews for bsEzTiVUU25IFCf9StjfVw.
Fetching Reviews for 71t6G6pw06d2zMpZw6sgzg.
Fetching Reviews for NGW5tqmDVJ9rtmNe7rF6cw.
Fetching Reviews for BveGHamJHQ8oB4tOPyc7fg.
Fetching Reviews for AefZD9E-mCB9HmTci8nRvQ.
Fetching Reviews for C_3v7HUO_K6OuqdYXC4msA.
Fetching Reviews for HnZDz0GpBQkzJ3YlgAva6A.
Fetching Reviews for sbJCvMg1r6J_d2twONWHSw.
Fetching Reviews for Q6TVsuMpXal1QNB-mTo4hw.
Fetching Reviews for Rf4YRfMucKuQISlPmt8jvA.
Fetching Reviews for X2qDSV0YVl5csxM4IT0t_Q.
Fetching R

Fetching Reviews for SiQpkGxvnA6IdCZwdhPnVg.
Fetching Reviews for o4Kl7EcgwncjKYnb9KORJA.
Fetching Reviews for I36TSHQGbPPr_kBSVLGskg.
Fetching Reviews for FT5tsPL6SEj6miLGdBqVDQ.
Fetching Reviews for Ewb83VswnuWlCsQF-tR0MQ.
Fetching Reviews for gode-5FIHnnsdaYjPWJpYw.
Fetching Reviews for moPCEnfT3IZc4riHOwczxQ.
Fetching Reviews for ojleeDZsyoRpUSgQwPQ7DA.
Fetching Reviews for J3ge7X1nTqJIFzk2BQFK5g.
Fetching Reviews for hHy4VuzqpYEZjRjCkRuQXg.
Fetching Reviews for 4EBFLCjAPwOX8IWVngnh5Q.
Fetching Reviews for qad3TzSFojlJ2Nom59Vf9w.
Fetching Reviews for DuthHu0LNruydNKkvleHMw.
Fetching Reviews for wchKJplrf_2ADixCZU3HOQ.
Fetching Reviews for lu7intkiJ0JnpV3inKhd1A.
Fetching Reviews for s8v3pC51Lf96I8PY8ajMbA.
Fetching Reviews for UsETmhz4H63QEUYppqWltQ.
Fetching Reviews for I_UlEi_OEIMTH0gelUmrNw.
Fetching Reviews for xWIxJOLgkwDWA-QSBpZ7cg.
Fetching Reviews for ccWGKiaaRlwq2lCbb5jBfw.
Fetching Reviews for TiO8G8Rx1b1uMULVQokIqw.
Fetching Reviews for WURtHYuN0-Q1Mm2jMgyeOw.
Fetching R

...

Fetching Reviews for DI395JHkn_9SDkUFsqeBjw.
Fetching Reviews for hYE3kXhFzREV2NuRwW7YJw.
Fetching Reviews for cV7Y1hdX5vJi-ruvKwM8cQ.
Fetching Reviews for i5BXcycBiEraUmbDD6XIng.
Fetching Reviews for XpaCBopSeoDeVW13SwqYtg.
Fetching Reviews for 6ugo61fTPk2IHV3Xwe5Hng.
Fetching Reviews for CKEUY8guO0g3kshZdLADdA.
Fetching Reviews for o88VvwDAX1oBrrKfZ7_iOQ.
Fetching Reviews for 9y8q6cREm44cvlTDh5jh6A.
Fetching Reviews for NemH3oTh27UZqKIWugiJGQ.
Fetching Reviews for QQ1I9-LkhywIWKicFQtaZQ.
Fetching Reviews for vd1sHX2Hf0adHMnfVz4Okw.
Fetching Reviews for V_fBfWukv1yFpHYZlHuthA.
Fetching Reviews for z8NGYKb6bVKBoD_JXF1ppg.
Fetching Reviews for 9nG3bqCCgtyXHs4cDAijhg.
Fetching Reviews for KFfArdu0kvnkfEFYanLJOA.
Fetching Reviews for z1XS739HSARpzIX-iG3Dag.
Fetching Reviews for QIb0Z3vN7x2-jBiyftyP2A.
Fetching Reviews for pxwzu0o9SWsjX1DAh3Z_HQ.
Fetching Reviews for 18dPynR7oUOfF0hoiNQd0A.
Fetching Reviews for XJYeoc8fP3relNLXY3zMZw.
Fetching Reviews for MlrqbJcFRTk7W-TwtiZpXQ.
Fetching R

Fetching Reviews for PoRCD7DoFBC3B2VbV8MXyg.
Fetching Reviews for x5QJqA3DpAROrTWiXTLUQQ.
Fetching Reviews for d0hBGo6iXDfQipAHg9QqNw.
Fetching Reviews for syxBtV8fk3ObUlMEcIqhBw.
Fetching Reviews for EwMqwS9LCOsoYxMdX8780A.
Fetching Reviews for KiNCDmo-ftTvapDpjF_WLw.
Fetching Reviews for _rTiztD_QLISoWXQFhs6Zg.
Fetching Reviews for dAeDb8YAhelvzN8pL09Cog.
Fetching Reviews for 7fHKnNfgHeNKybAtIsCUJw.
Fetching Reviews for enS7kydTL7tQ_tLuA9ctSg.
Fetching Reviews for KNBr2aS-GOcMhvVh5tm-tw.
Fetching Reviews for pcEZkz5onPNckAn0QxhKxw.
Fetching Reviews for -yG3-zzwQ4lZi6-iw1urwg.
Fetching Reviews for X7o4hkDJ2ejgK9mEcBK-UA.
Fetching Reviews for y6kX2zLjXDfgVWsNnsIsxQ.
Fetching Reviews for BpZvCEu_aP9XS4qGkjWgEA.
Fetching Reviews for YQuJfMkh7DsLoqL3H0ai7A.
Fetching Reviews for svd0O9QTiaibYaIpXpWecw.
Fetching Reviews for aWr0tJaEVY_idfLDx602wQ.
Fetching Reviews for P5_ooFqq8xde4hcD23EOhA.
Fetching Reviews for HWRFKJkZCKqFi800gxK2Ag.
Fetching Reviews for NRXA1WnGnCxXdzYcYdnz6w.
Fetching R

Fetching Reviews for oLIQ7cJXitiU59eymGBkBg.
Fetching Reviews for HPoxThtpnRI3zH2GlKmjSg.
Fetching Reviews for Q6uWrUdKZCbysdxmtD_2MQ.
Fetching Reviews for TcfudSnsiyFZSj0vKrlblQ.
Fetching Reviews for dMpyRJqmaTAZ-qCNhhowkA.
Fetching Reviews for XyuqcywRUm8_m1nHK6bEUw.
Fetching Reviews for w90LJ9VKVNt8IWr41fNFuw.
Fetching Reviews for oEOfuvKxiDlEq9xc-OY68A.
Fetching Reviews for nUAqiVc1gzFP6ArpoeNDCA.
Fetching Reviews for Kfo5ndtL0N9Fs9gAdX3zRA.
Fetching Reviews for KvAwvms8TGJXKU6SKPuJHw.
Fetching Reviews for EOxggoAthBc4Unz_CdI9Ug.
Fetching Reviews for cfsFYwJRjOmSLLbLF-_AAw.
Fetching Reviews for zUFPuntfahA3QelmaBlGtA.
Fetching Reviews for Y1DQwp8k5MrNmKfT114Zdg.
Fetching Reviews for nMw1xuxbZqG_YV5RH9h3xA.
Fetching Reviews for ngeP7ifx8W6R9G7o5v7OJA.
Fetching Reviews for WJEAMT8_uHA73MpG0MHUog.
Fetching Reviews for bBtRVWqV-vRFAClVt0J3gQ.
Fetching Reviews for Ex-N9-94TU__LCvsWHqOdw.
Fetching Reviews for a1x2c9H88Hb1yENfFSd75A.
Fetching Reviews for 0YWEXPcuq4NP5nXxskQxZg.
Fetching R

Fetching Reviews for 6Nrs2gflTpDH86ghgSqJ0Q.
Fetching Reviews for w3iw6P8Wbg-PEJLxFyE7sw.
Fetching Reviews for Dmmy0tHjbYP5pjqufG5suQ.
Fetching Reviews for OIoi-pnjpJoGU462yH0RyA.
Fetching Reviews for ERwalwWfC8viwEaWrokuBA.
Fetching Reviews for 8zDzZOG8ssWhlzawZy9n6Q.
Fetching Reviews for 2ioahRk9DTzNel-hexVWsw.
Fetching Reviews for jKvVX_xUKaT6Hs8Ypfo2DA.
Fetching Reviews for DUD6J2EfBgtYSheo1fdelQ.
Fetching Reviews for tAzoQ_eDjo2RHXc24MVGWQ.
Fetching Reviews for es8zrUMx-3TH3L5a9fUibw.
Fetching Reviews for c7HhEYMxlxbu2YPzw_LEsA.
Fetching Reviews for 7PhP8kS7zPa-af8wiOm3Ow.
Fetching Reviews for y6aDtLlirWXbFKEYiozdKw.
Fetching Reviews for dHPBFsoHJLTy38MuA2APng.
Fetching Reviews for mn-whTe5sxiIHuysmg8oWw.
Fetching Reviews for F3z2CnuZYdus0Kjb1qXEnA.
Fetching Reviews for BKDyvykoH_XgSK9vrET0qA.
Fetching Reviews for 0_xRhmUK8mSiMKlsTC74SQ.
Fetching Reviews for p6QAG6a5N_NGYRCnJlM5Ew.
Fetching Reviews for eBsOIMKYz449LLmEFKnyBg.
Fetching Reviews for k2rzpRVqg1VFZqrnJ9ywOA.
Fetching R

Fetching Reviews for CathvT3S1XqJ-Kgv6lW7Ow.
Fetching Reviews for WjOVYp_ynZHjnlTPZJznhA.
Fetching Reviews for dmvhlUKdTTRHmvC14tbaFg.
Fetching Reviews for GoyFXv1reRzaYUztoQyV-g.
Fetching Reviews for LlUSYGupMRI2uPcp5WQxyA.
Fetching Reviews for Wnkgilydeo5qZg4SXZqMxw.
Fetching Reviews for IMeYJItLTeOuDNdLPzcmUA.
Fetching Reviews for 7UAQ7DjYvt4cMr1fblWzVQ.
Fetching Reviews for UeC2HiYZZ5AOeAE8xbGg9g.
Fetching Reviews for QH5h3TJWI3uRWavkw_Fw6A.
Fetching Reviews for _AlNjRlsKP-yniO4Mdw1Bg.
Fetching Reviews for TqrmYNvl8OkDlCC1HADsDQ.
Fetching Reviews for JFKv8t8I5Wm5mtKhzDtOHA.
Fetching Reviews for 02qDQOwhVI1ydF7zM2eJ9Q.
Fetching Reviews for hIfDV6211r3yHHooXL68Dw.
Fetching Reviews for QVMOq3JukHS1jQIbGMj9UA.
Fetching Reviews for ngwc_s-Wx23xlZch6a5cAg.
Fetching Reviews for IfpZ4hTkm5cvdAvpeVLwlw.
Fetching Reviews for eb_Dvm7SHeVn5mABo73ZWA.
Fetching Reviews for 6hx2Gu0TqhQGgYNdtEpYZA.
Fetching Reviews for qnwSjl5X5BQqZbbHbIN1Pw.
Fetching Reviews for xJgGxvKMhdQDzXBaJDasQg.
Fetching R

In [25]:
graph_review=pd.DataFrame.from_dict(review_data, orient='columns', dtype=None, columns=None)

In [26]:
graph_review["Compound"]=""
graph_review.head()

for index, row in graph_review.iterrows():
    results = analyzer.polarity_scores(row["Review"])
    graph_review.loc[index, "Compound"] = results["compound"]
graph_review.head()

,Business ID,Review,Compound
0,LVdPec0oNU0xm5RBzcsDTQ,Closing with Attorney Goldberg was easy. He m...,0.8805
1,LVdPec0oNU0xm5RBzcsDTQ,Ken did an outstanding job helping us sell the...,0.7227
2,LVdPec0oNU0xm5RBzcsDTQ,Ken Goldberg is what all attorneys should stri...,0.8555
3,6gan8fxBSkGQJ7vLsK0ENQ,Had a great experience using Leppo & Leppo as ...,-0.0772
4,WcFu0Esq3cm-hKHichMcXg,5 stars because I want to make sure people see...,-0.7805


In [27]:
final=graph_review.merge(graph_data, on="Business ID", how="left")
final.head(100000)

,Business ID,Review,Compound,Business Name,Business Alias,Category,City,State,ZipCode,Lat,Long,Review Count,Rating
0,LVdPec0oNU0xm5RBzcsDTQ,Closing with Attorney Goldberg was easy. He m...,0.8805,Kenneth J Goldberg,kenneth-j-goldberg-brockton-2,lawyers,Brockton,MA,02301,42.073904,-71.046095,8,5.0
1,LVdPec0oNU0xm5RBzcsDTQ,Ken did an outstanding job helping us sell the...,0.7227,Kenneth J Goldberg,kenneth-j-goldberg-brockton-2,lawyers,Brockton,MA,02301,42.073904,-71.046095,8,5.0
2,LVdPec0oNU0xm5RBzcsDTQ,Ken Goldberg is what all attorneys should stri...,0.8555,Kenneth J Goldberg,kenneth-j-goldberg-brockton-2,lawyers,Brockton,MA,02301,42.073904,-71.046095,8,5.0
3,6gan8fxBSkGQJ7vLsK0ENQ,Had a great experience using Leppo & Leppo as ...,-0.0772,Leppo & Leppo,leppo-and-leppo-brockton,personal_injury,Brockton,MA,02301,42.054674,-71.060524,1,5.0
4,6gan8fxBSkGQJ7vLsK0ENQ,Had a great experience using Leppo & Leppo as ...,-0.0772,Leppo & Leppo,leppo-and-leppo-brockton,personal_injury,Brockton,MA,02301,42.054674,-71.060524,1,5.0
5,WcFu0Esq3cm-hKHichMcXg,5 stars because I want to make sure people see...,-0.7805,Curran Sheila T,curran-sheila-t-brockton-2,lawyers,Brockton,MA,02301,42.084435,-71.021690,1,5.0
6,6zj8agbUy2kdJElqSNProQ,Do not use this attorney. He has been suspende...,-0.4767,Carchidi Paul Atty,carchidi-paul-atty-brockton,lawyers,Brockton,MA,02301,42.076279,-71.032486,1,1.0
7,jYYzEiMr-8SZqXzFVjd2wg,My husband and I are building a new home and J...,0.6486,Moriarty Jonathan J,moriarty-jonathan-j-brockton,lawyers,Brockton,MA,02301,42.054660,-71.060480,1,5.0
8,jYYzEiMr-8SZqXzFVjd2wg,My husband and I are building a new home and J...,0.6486,Moriarty Jonathan J,moriarty-jonathan-j-brockton,lawyers,Brockton,MA,02301,42.054660,-71.060480,1,5.0
9,_KOFlsdPKZ7eFG3gBgEhzQ,This guy is a dingbat. There is no one answeri...,-0.296,Law Office of Thomas A Dougherty III,law-office-of-thomas-a-dougherty-iii-brockton,lawyers,Brockton,MA,02301,42.082744,-71.019787,1,1.0
